In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Sample retail data (replace with your own retail data)
data = {
    'CustomerID': [1, 2, 3, 1, 2],
    'ProductID': ['A', 'B', 'A', 'B', 'C'],
    'PurchaseCount': [1, 1, 2, 1, 3]
}

df = pd.DataFrame(data)

# Encode customer and product IDs
encoder = LabelEncoder()
df['CustomerID'] = encoder.fit_transform(df['CustomerID'])
df['ProductID'] = encoder.fit_transform(df['ProductID'])

# Create a user-item matrix
user_item_matrix = df.pivot(index='CustomerID', columns='ProductID', values='PurchaseCount').fillna(0)

# Calculate item-item similarity (cosine similarity)
item_similarity = cosine_similarity(user_item_matrix.T)

# Create a dataframe of item similarity scores
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Function to recommend items to a user
def recommend_items(user_id, num_recommendations=5):
    user_purchase_history = user_item_matrix.iloc[user_id]
    user_unpurchased_items = user_purchase_history[user_purchase_history == 0].index

    # Calculate a weighted score for each unpurchased item
    weighted_scores = item_similarity_df.loc[user_unpurchased_items].dot(user_purchase_history)

    # Sort items by score in descending order
    recommended_items = weighted_scores.sort_values(ascending=False)

    # Get the top 'num_recommendations' items
    top_recommendations = recommended_items.head(num_recommendations)

    return top_recommendations.index

# Example: Recommend 3 items to user 0
user_id = 0
recommended_items = recommend_items(user_id, num_recommendations=3)

print(f"Recommended items for user {user_id}:")
for item_id in recommended_items:
    print(f"Item {encoder.inverse_transform([item_id])[0]}")


Recommended items for user 0:
Item C
